In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math

In [21]:
df = pd.read_csv("./dataset/20170827_mawilab_flow_000.csv",sep=',')
df.shape 

(383615, 32)

In [22]:
df.loc[:, 'U'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
df.loc[:, 'N'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
new_flag_df = df['flags'].to_frame()
new_flag_df = new_flag_df['flags'].str.get_dummies(' ')
new_flag_df.head()

,A,C,E,EC,F,PA,R,S
0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


In [23]:
"""
1. Pro
2. Packets
3. Bytes
4. Flags
5. Durat
6. Records
7. svc_port - combination of sPort and dPort

v = np.minimum(df['sPort'], df['dPort'])
df['svc_port'] = np.where(v <= 1024, v, df['dPort'])
"""
df['svc_port'] = df.apply(lambda x : min(x['sPort'],x['dPort']) if min(x['sPort'],x['dPort']) <= 1024 else x['dPort'], axis=1)
df = df.loc[:,('pro', 'packets', 'bytes', 'svc_port', 'durat','U','N','Records')]
df.fillna(0, inplace=True) 
new_df = df.drop(['Records'], axis=1)
mawii_labels = pd.get_dummies(df['Records'],prefix='',prefix_sep='')
mawii_features = pd.concat([new_df, new_flag_df], axis=1)
print(mawii_labels.head())
print(mawii_features.head())

   anomaly  normal
0        0       1
1        0       1
2        0       1
3        1       0
4        1       0
   pro  packets  bytes  svc_port  durat  U  N  A  C  E  EC  F  PA  R  S
0    1        1     32         0    0.0  0  0  0  0  0   0  0   0  0  0
1    6        1     48        80    0.0  0  0  1  0  0   0  0   0  0  1
2    6        1   1440        22    0.0  0  0  1  0  0   0  0   0  0  0
3    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0
4    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0


In [24]:
#Test set preparation


In [29]:
mawii_features.shape

(383615, 15)

In [42]:
from keras.models import Sequential
from keras.layers import Conv1D,Dense, MaxPooling2D,Flatten
import keras
input_shape = mawii_features.shape
num_classes = 2

# ConvNet_1
model = Sequential() 
model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),activation='relu',input_shape=(128,15,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

ValueError: The `kernel_size` argument must be a tuple of 1 integers. Received: (5, 5)

In [41]:
model.fit(mawii_features, mawii_labels,
          batch_size=128,
          epochs=1000,
          verbose=1,
          validation_data=(mawii_features, mawii_labels))

ValueError: Error when checking input: expected conv2d_12_input to have 4 dimensions, but got array with shape (383615, 15)

In [3]:
def conv_2d(x,W): 
    # Padding : surround the original image with 0's 
    # valid padding - no padding.size of the image shrinks  resultant image : (n - f + 1, n - f + 1)
    # same padding - resultant is same as that of original. resultant image : (n + 2p - f + 1, n +2p - f + 1)  p = (f - 1)/2
    # Strides : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.conv2d(input = x, filter = W, strides = [1,1,1,1], padding="SAME")

def avg_pool_2x2(x):

    # MAX/AVG Pooling : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.avg_pool(x, k_size=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [ ]:
# cnn-1,2,3 model 

In [ ]:
def conv1_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet1') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Fully Connected layer1 
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [16], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool1, [-1,7*7*8])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [16,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv 

In [ ]:
def conv2_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet2') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [32], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool2, [-1,7*7*16])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [32,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv
        

In [ ]:
def conv3_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet3') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # Third Conv and Pooling layers
        W_conv3 = tf.get_variable('d_wconv2',[5,5,16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv3 = tf.get_variable('d_bconv2',[32],initializer=tf.constant_initializer(0))
        h_conv3 = tf.nn.relu(conv_2d(h_pool2, W_conv3) + b_conv3)
        h_pool3 = avg_pool_2x2(h_conv3)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*32,64],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [64], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool3, [-1,7*7*32])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [64,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv